Copyright © 2025 HQS Quantum Simulations GmbH. All Rights Reserved.

# Elecron spectroscopy: singlet-triplet splitting with read-in Hamiltonian

In [1]:
# HQS
from hqs_quantum_solver.spins import VectorSpace, Operator
from hqs_quantum_solver.spins import struqture_term
from struqture_py.spins import PauliHamiltonian

# General
import json
import numpy as np
import scipy.sparse as sparse
import matplotlib.pyplot as plt
from scipy.sparse.linalg import eigsh

# License check turned off for lattice_functions.
# License check turned off for lattice_solver.


Load a previously generated 4-qubit Hamiltonian from .json-file

In [2]:
ham_json = "cyclopentadiene_4_qubits.json"
with open(ham_json, "r", encoding="utf-8") as f:
    ham = PauliHamiltonian.from_json(f.read())

Build and solve Hamiltonians with HQS_quantum_solver

In [3]:
def build_and_solve(num_spins_bath, ham):
    num_spins_cas = 2
    # vector space for the active orbital (CAS) space
    v_cas = VectorSpace(sites=num_spins_cas, total_spin_z="all")
    # vector spaces for the bosonic bath for empty, single and double excitations in the bath
    v_bath_0 = VectorSpace(sites=num_spins_bath, total_spin_z=-num_spins_bath)
    v_bath_1 = VectorSpace(sites=num_spins_bath, total_spin_z=-num_spins_bath + 2)
    v_bath_2 = VectorSpace(sites=num_spins_bath, total_spin_z=-num_spins_bath + 4)
    v_bath_3 = VectorSpace(sites=num_spins_bath, total_spin_z=-num_spins_bath + 6)
    v_bath_4 = VectorSpace(sites=num_spins_bath, total_spin_z=-num_spins_bath + 8)
    #  Now build the complete vector space
    v_bath = v_bath_0 | v_bath_1 | v_bath_2 | v_bath_3 | v_bath_4  # | v_bath_5
    v = v_cas * v_bath
    # Convert struqture Hamiltonian directly to Quantum Solver operator
    H = Operator(struqture_term(ham), domain=v)
    # Now solve as usual
    k = min(10, H.shape[0] - 1)
    eigenvalues, _ = eigsh(H, k=k, which="SA")

    return eigenvalues

Singlet triplet gap $\Delta E_{ST}$ is a difference between first excited state energy (triplet) and ground state energy (singlet).

In [4]:
num_spins_bath = 4  # We know that Hamiltonian includes 4 qubits
print("Nq = ", num_spins_bath)
eigenvalues = build_and_solve(num_spins_bath=num_spins_bath, ham=ham)
print("Singlet triplet gap:")
print("E_t-E_g=", (eigenvalues[1] - eigenvalues[0]) * 27.2114, "eV")

Nq =  4
Singlet triplet gap:
E_t-E_g= 3.394010755984705 eV
